( klines['Close'].max() / klines['Close'].min() ) range for how much it has fluctuated before buying, if it has gone down alot or already gone up alot

In [1]:
from sys import displayhook
from binance.client import Client
from binance.enums import *
import pandas as pd
import time
import datetime
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt

def getminutedata(symbol, start, end):
    frame = pd.DataFrame(client.get_historical_klines(symbol, Client.KLINE_INTERVAL_15MINUTE, start, end))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame.reset_index()

api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)

In [2]:
api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)
db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)

check_date = pd.read_sql_query("SELECT MAX(date_request) FROM crypto_trend", engine) 
date_recent_checked = str(check_date.iloc[0, 0])

trend_table_status = pd.read_sql_query("show table status like 'crypto_trend'", engine)
trend_table_rowcount_old = trend_table_status['Rows'].values[0]

cryptoString = 'SELECT * FROM crypto'
crypto_table = pd.read_sql_query(cryptoString, engine)
full_id_list = crypto_table.id.astype(int).tolist()

KeyboardInterrupt: 

In [3]:
start_string_close = "2022-03-11 12:00:00"
start_string = "2022-03-11 14:00:00"
end_string = "2022-04-03 21:00:00"
start_date = datetime.datetime.strptime(start_string, '%Y-%m-%d %H:%M:%S')
end_date = datetime.datetime.strptime(end_string, '%Y-%m-%d %H:%M:%S')

new_close_string = f"SELECT crypto_id, date, close FROM crypto_price WHERE date >= '{start_string_close}' AND date <= '{end_string}' ORDER BY date"
closes_all = pd.read_sql_query(new_close_string, engine)

new_trends_string = f"SELECT * FROM crypto_trend WHERE date_request > '{start_string}' AND date_request < '{end_string}' ORDER BY date_request, crypto_id, date"
new_trends_all = pd.read_sql_query(new_trends_string, engine)

NameError: name 'engine' is not defined

In [125]:
def trade_sim(maxholds, buy_threshold, sell_threshold):
    #setup variables
    #maxholds = 3
    #buy_threshold = 80
    #sell_threshold = 50
    trades = [[],[],[],[]]
    tradedata = []
    buyPrice  = [0] * (len(full_id_list))
    sellPrice = [0] * (len(full_id_list))
    tradeBuy  = [0] * (len(full_id_list))
    tradeSell = [0] * (len(full_id_list))
    buyTime   = [0] * (len(full_id_list))
    sellTime  = [0] * (len(full_id_list))
    holding = [False] * (len(full_id_list))
    buy_ids = []

    for i in range(int((end_date - start_date).total_seconds()/3600)):
        date_lower = str(start_date + i*datetime.timedelta(hours=1))
        date_upper = str(start_date + (i+1)*datetime.timedelta(hours=1))
        date_price_lower = str(start_date + i*datetime.timedelta(hours=1) - datetime.timedelta(hours=5))
        ############################################ TESTING ONLY ##################################################################
        
        ##new_trends_string = f"SELECT * FROM crypto_trend WHERE date_request > '{date_lower}' AND date_request < '{date_upper}' ORDER BY date_request, crypto_id, date"
        ##new_trends = pd.read_sql_query(new_trends_string, engine)

        new_trends = new_trends_all.loc[(new_trends_all['date_request'] > date_lower) & (new_trends_all['date_request'] <= date_upper)]

        ####################################################################################################################################

        #update to ensure that if the count increased inbetween the request for count and request for data, does not re-check
        trend_table_rowcount_old = pd.read_sql_query("show table status like 'crypto_trend'", engine)['Rows'].values[0]

        if new_trends.empty:
            print(f"Dataframe empty for request: {new_trends_string}")
            check_data = False
            waiting = True
            
        date_request = new_trends[['crypto_id', 'date_request']].groupby(['crypto_id']).mean()
        date_recent_checked = new_trends.date_request.max()

        try:
            trends = new_trends.pivot(index="date", columns="crypto_id", values="hype") #unbeliiiiieveevevevable !! https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html
        except:
            pd.set_option("display.max_rows", None, "display.max_columns", None)
            print(new_trends)
            

        #check buying and selling conditions
        check_100 = ((trends.iloc[-2] == 100))# | (trends.iloc[-1] == 100)) ##check trends with current time
        check_non0 = trends.iloc[0:-2].mean() > 7 # 0:-2
        check_peak = (trends.iloc[0:-10] < buy_threshold).all()
        trend_indicate_buy = np.logical_and.reduce((check_100, check_non0, check_peak), dtype=bool)

        curr_id_list = trends.columns.values
        check_upward_ids = [curr_id_list[i] for i in [i for i, x in enumerate(trend_indicate_buy) if x]]


        buy_ids = []
        if check_upward_ids:
            #print(f"Indicate buy ids: {check_upward_ids} between {date_last_checked} : {date_recent_checked}")
            for key, crypto_id in enumerate(check_upward_ids):
                date_lower_str = str(date_request.loc[crypto_id]['date_request'] - datetime.timedelta(hours=2))
                date_upper_str = str(date_request.loc[crypto_id]['date_request'])
                #new_close_string = f"SELECT date, close FROM crypto_price WHERE crypto_id = {crypto_id} AND date > '{date_lower_str}' AND date <= '{date_upper_str}' ORDER BY date"
                #close = pd.read_sql_query(new_close_string, engine)
                close = closes_all.loc[ (closes_all['crypto_id'] == crypto_id) & (closes_all['date'] > date_lower_str) & (closes_all['date'] <= date_upper_str) ]       
                close.index = pd.to_datetime(close['date'])
                close = close.drop(columns=['date', 'crypto_id'])
                close = close.rename(columns={"close": f"{crypto_id}"})
                if (key != 0):
                    closes = closes.join(close)
                else:
                    closes = close

            increase = closes.diff(periods=6)
            check_increase = (increase.iloc[-1] >= 0)

            indicate_buy = check_increase[check_increase].index.values.astype(int)
            #print(f"Indicate buy ids: {indicate_buy} between {date_last_checked} : {date_recent_checked}")

            curr_NOT_holding_ids = [full_id_list[i] for i in [i for i, x in enumerate(np.logical_not(holding)) if x]]
            buy_ids = list(set(indicate_buy).intersection(curr_NOT_holding_ids))

        check_decrease = np.logical_or((trends.iloc[-1] < sell_threshold), (trends.iloc[-2] < sell_threshold))
        indicate_sell = check_decrease[check_decrease].index.values.astype(int)
        #print(f"Indicate sell ids: {indicate_sell} between {date_last_checked} : {date_recent_checked}")

        curr_holding_ids = [full_id_list[i] for i in [i for i, x in enumerate(holding) if x]]
        sell_ids = list(set(indicate_sell).intersection(curr_holding_ids))

        # Buy control
        if ((sum(holding) < maxholds) and buy_ids):
            for id in buy_ids:
                if (sum(holding) < maxholds):
                    key = full_id_list.index(id)
                    tradetime = closes.index[-1]
                    buyPrice[key] = closes.loc[tradetime,str(id)]
                    buyTime[key] = tradetime
                    holding[key] = True
                    #print(f"Purchased {crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0]} id {id}  at {buyPrice[key]} - {tradetime}")
                else:
                    continue
                
        # Sell control
        if (sell_ids):
            for id in sell_ids:
                date_lower_str = str(date_request.loc[crypto_id]['date_request'] - datetime.timedelta(hours=2))
                date_upper_str = str(date_request.loc[crypto_id]['date_request'])
                #new_close_string = f"SELECT date, close FROM crypto_price WHERE crypto_id = {id} AND date > '{date_lower_str}' AND date <= '{date_upper_str}' ORDER BY date"
                #close = pd.read_sql_query(new_close_string, engine)       
                close = closes_all.loc[ (closes_all['crypto_id'] == id) & (closes_all['date'] > date_lower_str) & (closes_all['date'] <= date_upper_str) ]                                
                close.index = pd.to_datetime(close['date'])
                close = close.drop(columns=['date', 'crypto_id'])
                # Sell execute function binance using ticker
                key = full_id_list.index(id)
                tradetime = close.index[-1]
                sellPrice[key] = close.loc[tradetime, 'close']
                sellTime[key] = tradetime
                #print(f"Trade completed: {crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0]} id = {id} | open = {buyPrice[key]} at {buyTime[key]} | close = {sellPrice[key]} at {sellTime[key]} | profit = {(sellPrice[key]/buyPrice[key])}%")

                # upload to database
                quantity = 0
                trade = (id, buyTime[key], buyPrice[key], sellTime[key], sellPrice[key], str(sellTime[key] - buyTime[key]),  sellPrice[key]/buyPrice[key], quantity)
                index_names = ['crypto_id', 'date_buy', 'price_buy', 'date_sell', 'price_sell', 'trade_time', 'trade_return', 'quantity']
                result = pd.DataFrame([trade],columns=index_names)
                #print(result)
                tradedata.append([id, buyTime[key], sellTime[key], buyPrice[key], sellPrice[key], (sellPrice[key]/buyPrice[key]) ])
                holding[key] = False
    return tradedata

first test = 0.8694298637673121 (not equal to 0.65)
Changing time to time of trend. (just realised this was scaled and not actual, assume if it is scaled it is closer to 1 and less frequent big wins / losses are les impactful ?)
Second test = 0.6355674133307827, 0.8694298637673121 scalled (worked first time lol)

In [121]:
with open('output_results_180322.pickle', 'rb') as handle:
    output_results_180322_in = pickle.load(handle)
valid_cond = [x[0] for x in output_results_180322_in if x[1].size != 0]

In [63]:
import itertools
iterables = [ range(10,100,10), range(10,100,10) ]
output = []
output_results = []
for t in itertools.product(*iterables):
    maxholds = 3
    buy_threshold = t[0]
    sell_threshold = t[1]

    results = trade_sim(maxholds, buy_threshold, sell_threshold)
    results = np.array(results)
    #gain = results[:,5].prod()
    #output.append([t, gain])
    output_results.append([t, results])
    

In [126]:
output_results = []
for t in valid_cond:
    maxholds = 3
    buy_threshold = t[0]
    sell_threshold = t[1]

    results = trade_sim(maxholds, buy_threshold, sell_threshold)
    results = np.array(results)
    #gain = results[:,5].prod()
    #output.append([t, gain])
    output_results.append([t, results])

In [127]:
# import pickle
# with open('output_results_180322_not_partial.pickle', 'wb') as handle:
#     pickle.dump(output_results, handle)

In [137]:

for x in output_results:
    if x[1].size == 0:
        output_results.remove(x)

#gains = 

In [146]:
gains = [(x[0], x[1][:,5].mean()) for x in output_results]
sorted(gains, key = lambda x : x[1])

[((60, 30), 0.9753105697688287),
 ((60, 20), 0.9838289789566218),
 ((60, 50), 0.9888867669070305),
 ((60, 40), 0.9893193864181705),
 ((70, 20), 0.9944073687074995),
 ((70, 30), 0.9948915328398956),
 ((50, 20), 0.9971209213051824),
 ((90, 70), 0.9975543422765988),
 ((60, 70), 0.9980907530466804),
 ((70, 50), 0.9981254243317094),
 ((70, 40), 0.9983035617774729),
 ((60, 80), 0.9984306683768617),
 ((80, 30), 0.9988693168470208),
 ((80, 70), 0.9989824328481998),
 ((80, 50), 0.9993061576233673),
 ((60, 90), 0.9994195129737531),
 ((50, 30), 1.0),
 ((50, 40), 1.0),
 ((50, 50), 1.0),
 ((50, 60), 1.0),
 ((50, 70), 1.0),
 ((50, 80), 1.0),
 ((50, 90), 1.0),
 ((90, 80), 1.000670883214151),
 ((90, 50), 1.0010178727963457),
 ((90, 90), 1.0011644878233834),
 ((90, 60), 1.0016961886441544),
 ((50, 10), 1.0023992322456814),
 ((90, 40), 1.0025932293406872),
 ((80, 20), 1.0027839609204043),
 ((80, 80), 1.003206861763829),
 ((80, 40), 1.0033703234283773),
 ((70, 70), 1.0035983776415893),
 ((80, 90), 1.0042

In [143]:
trade_data_27[:,5].prod()

1.417520849740263

In [140]:
trade_data_27 = output_results[27][1]

In [142]:
(((trade_data_27[:,5] - 1) * 1/maxholds) + 1).prod()

1.1333708818952664

In [144]:
td27 = pd.DataFrame(trade_data_27)

In [145]:
pd.set_option('display.max_rows', 500)
td27.sort_values(by = [5], ascending=False)


,0,1,2,3,4,5
11,170,2022-03-14 01:00:00,2022-03-14 22:00:00,0.01315,0.016399,1.247072
0,235,2022-03-11 22:00:00,2022-03-11 23:00:00,5.219,5.838,1.118605
5,65,2022-03-12 12:00:00,2022-03-12 14:00:00,11.92,13.25,1.111577
15,284,2022-03-16 06:00:00,2022-03-16 09:00:00,8.11,8.26,1.018496
7,65,2022-03-12 21:00:00,2022-03-12 22:00:00,12.29,12.4,1.00895
8,288,2022-03-13 02:00:00,2022-03-13 03:00:00,0.4152,0.417,1.004335
14,36,2022-03-15 23:00:00,2022-03-16 00:00:00,7.09,7.11,1.002821
3,131,2022-03-12 01:00:00,2022-03-12 06:00:00,0.2084,0.2089,1.002399
6,192,2022-03-12 14:00:00,2022-03-12 17:00:00,0.0719,0.0719,1.0
13,152,2022-03-15 05:00:00,2022-03-15 22:00:00,0.4623,0.4622,0.999784


In [ ]:
pd.DataFrame(output_results[27][1]).sort_values(by = [5], ascending=False)